***Webscraper Hotels Data - 123 Milhas / Decolar***

*   Data extraction name, price and provider hotels website 123 Milhas
*   Data extraction name and price hotels website Decolar
*   Report rate comparison

In [1]:
!pip install selenium
!pip install unidecode
!pip install pyjarowinkler
!apt-get update 
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from unidecode import unidecode
from time import sleep
import pandas as pd
import os

In [3]:
# ID destinations

# 123 Milhas
  # Aracaju: 1010804
  # Arraial dAjuda: 1184784
  # Balneario Camboriu: 1013062
  # Belo Horizonte: 1017081
  # Brasilia: 1262440
  # Buzios: 1156720
  # Cabo Frio: 1027418
  # Caldas Novas: 1027035
  # Campinas: 1265185
  # Campos do Jordao: 1621906
  # Canela: 1006401
  # Curitiba: 1112002
  # Florianopolis: 1001872
  # Fortaleza: 1001925
  # Foz do Iguacu: 1106867
  # Gramado: 1006404
  # Ilhabela: 1012575
  # Ilheus: 1002846
  # Joao Pessoa: 1111236
  # Maceio: 1009657
  # Morro de Sao Paulo: 1060292
  # Natal: 1009933
  # Porto Alegre: 1112012
  # Porto de Galinhas: 1024069
  # Porto Seguro: 1262144
  # Praia de Pipa: 1060267
  # Recife: 1010394
  # Rio de Janeiro: 1010422
  # Salvador: 1010693
  # Sao Paulo: 1010502

# Decolar
  # Aracaju: 180
  # Arraial dAjuda: 339
  # Balneario Camboriu: 192594
  # Belo Horizonte: 701
  # Brasilia: 926
  # Buzios: 1077
  # Cabo Frio: 1227
  # Caldas Novas: 1364
  # Campinas: 1467
  # Campos do Jordao: 6116
  # Canela: 6064
  # Curitiba: 1595
  # Florianopolis: 2261
  # Fortaleza: 2302
  # Foz do Iguacu: 3072
  # Gramado: 2611
  # Ilhabela: 3027
  # Ilheus: 3158
  # Joao Pessoa: 3399
  # Maceio: 4430
  # Morro de Sao Paulo: 4678
  # Natal: 4971
  # Porto Alegre: 5822
  # Porto de Galinhas: 5648
  # Porto Seguro: 881
  # Praia de Pipa: 5875
  # Recife: 6322
  # Rio de Janeiro: 6381
  # Salvador: 7018
  # Sao Paulo: 6574

In [4]:
# Search params

name_destination = 'Florianopolis'
id_destination_123milhas = '1001872'
id_destination_decolar = '2261'
checkin = '10-03-2022' # Format DD-MM-AAAA
checkout = '12-03-2022' # Format DD-MM-AAAA
pax = 2 # Number of guests (adults)

In [5]:
# Chromedriver config

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
# Get fee 123 Milhas

def get_fee():
    driver = webdriver.Chrome(options=chrome_options)
    link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
    driver.get(link)
    sleep(3)
       
    price_net_temp = []
    tax_temp = []

    wait = WebDriverWait(driver, 10)

    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'image-slider__main-image')))        
    hotel1_attributes = driver.find_element(By.XPATH, '/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[1]/div[2]/div[3]/div[4]/theme-button/button').get_attribute('outerHTML')
    attributes_filter1 = ''.join(hotel1_attributes.partition('id')[1:])
    attributes_filter2 = attributes_filter1.find('type')
    attributes_filter3 = attributes_filter1[:attributes_filter2]
    attributes_replacers = attributes_filter3.replace('"','').replace('=','-').replace(' ','').replace('id-','')
    query = "'#" + attributes_replacers + "'"
    
    driver.execute_script(f"document.querySelector({query}).click();")
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)
    
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'rooms-details-and-price__button')))
    driver.execute_script("document.querySelector('#hotel-details-and-info > div > rooms-details-and-price > div > div.rooms-details-and-price__prices-select-holder > theme-button > button').click();")
    
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-checkout-prices-and-tax')))        
    price_net_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[2]/span[2]')
    tax_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[3]/span[2]')

    price_net_temp.append(price_net_elem.text)
    tax_temp.append(tax_elem.text)
    price_net = rate_fmt(price_net_temp)
    tax = rate_fmt(tax_temp)
    
    global fee
    calc_fee = tax / price_net
    fee = float("%.4f" % calc_fee)
    return fee


def rate_fmt(rate):
    rate = ' '.join(rate)
    rate = rate.strip('R$').lstrip().replace(',','.')
    rate = float(rate)
    return rate



In [7]:
print('Fee 123 Milhas:')
get_fee()

Fee 123 Milhas:


0.1446

In [8]:
# Scraper Hotel Prices 123 Milhas

list_name_hotels_123milhas = []
list_price_hotels_123milhas = []
list_channel_hotels_123milhas = []

class Scraper123milhas:

    def scraper(self):
        self.data_collect()

    def data_collect(self):
        self.list_names = []
        self.list_prices = []
        self.list_channels = []
        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
        self.driver.get(self.link)
        sleep(3)
        wait = WebDriverWait(self.driver, 10)

        for p in range(20): # Number of pages to browse (nº > total, run through all)
            item = 1
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-list-card__total-price')))
            list_length = self.driver.find_elements(By.CLASS_NAME, 'hotel-list-card__total-price')
            list_max = len(list_length)
            for i in range(list_max):
                c = 1
                while c < list_max:
                    try:
                        names_temp = self.driver.find_elements(By.XPATH,
                            f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/div/h4')
                        self.list_names.append(names_temp[0].text)                       
                        sleep(1)
                        prices_temp = self.driver.find_elements(By.XPATH,
                            f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/div[1]/div/div')
                        self.list_prices.append(prices_temp[0].text)          
                        sleep(1)
                        channels_temp = self.driver.find_element(By.XPATH,
                            f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/image-slider/div/div[1]/img').get_attribute('outerHTML')
                        if len(channels_temp) == 0:
                            channels_temp = self.driver.find_element(By.XPATH, f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/image-slider/div/div[1]/img').get_attribute('outerHTML')                        
                        if 'omnibees' in channels_temp:
                            self.list_channels.append(1)
                        else:
                            self.list_channels.append(0)
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1                        
            try:
              wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'pagination-next')))
              self.driver.execute_script('document.querySelectorAll("#hotelListHolder > div.hotel-card-price-list__cards-holder.hotel-card-price-list__obj-align > div.hotel-card-price-list__pagination-container > ul > li.pagination-next > a")[0].click();')
              print(f'\u001b[32m{"Browsing..."}\u001b[0m')
              sleep(3)
            except:
              self.driver.quit()

        print('Raw Data:')
        print(self.list_names)
        print(self.list_prices)
        print(self.list_channels)

        for name in self.list_names:
            list_name_hotels_123milhas.append(name)
        for price in self.list_prices:
            list_price_hotels_123milhas.append(price)  
        for channel in self.list_channels:
            list_channel_hotels_123milhas.append(channel)

start = Scraper123milhas()
start.scraper()


Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Raw Data:
['Costa Norte Ponta das Canas', 'Hotel Porto Sol Ingleses', 'Hotel Cecomtur Florianópolis - Rede Andrade', 'Hotel Intercity Portofino Florianópolis', 'Hotel Valerim Florianópolis', 'Slaviero Baía Norte Florianópolis', 'Faial Prime Suites', 'Castelmar Hotel', 'Blue Tree Premium Florianópolis', 'Novotel Florianópolis', 'Cris Hotel', 'Village Paraíso', 'Lumar Hotel - Rede Andrade', 'Gentil Hotel', 'Palace Praia Hotel', 'Villas Jurerê Hotel Boutique', 'Hotel Intercity Florianópolis', 'Pousada Bizkaia', 'Ingleses Praia Hotel', 'Porto da Ilha Hotel', 'Costa Norte Ingleses', 'Interclass Florianópolis', 'Iate Hotel Centro Florianópolis', 'Rio Branco Apart Hotel', 'Varadero Palace Hotel II', 'Hotel Porto Sol Beach', 'Maria do Mar Hotel', 'Hotel Bout

In [9]:
# Data cleaning

name_hotels_123m_temp = list(pd.Series(list_name_hotels_123milhas).str.upper())
name_hotels_123m_fmtg = []

for name in name_hotels_123m_temp:
    name_hotels_123m_fmtg.append(unidecode(name))
print(name_hotels_123m_fmtg)

['COSTA NORTE PONTA DAS CANAS', 'HOTEL PORTO SOL INGLESES', 'HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE', 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS', 'HOTEL VALERIM FLORIANOPOLIS', 'SLAVIERO BAIA NORTE FLORIANOPOLIS', 'FAIAL PRIME SUITES', 'CASTELMAR HOTEL', 'BLUE TREE PREMIUM FLORIANOPOLIS', 'NOVOTEL FLORIANOPOLIS', 'CRIS HOTEL', 'VILLAGE PARAISO', 'LUMAR HOTEL - REDE ANDRADE', 'GENTIL HOTEL', 'PALACE PRAIA HOTEL', 'VILLAS JURERE HOTEL BOUTIQUE', 'HOTEL INTERCITY FLORIANOPOLIS', 'POUSADA BIZKAIA', 'INGLESES PRAIA HOTEL', 'PORTO DA ILHA HOTEL', 'COSTA NORTE INGLESES', 'INTERCLASS FLORIANOPOLIS', 'IATE HOTEL CENTRO FLORIANOPOLIS', 'RIO BRANCO APART HOTEL', 'VARADERO PALACE HOTEL II', 'HOTEL PORTO SOL BEACH', 'MARIA DO MAR HOTEL', 'HOTEL BOUTIQUE QUINTA DAS VIDEIRAS', 'OSCAR HOTEL', 'HOTEL PALACE 1', 'POUSADA GOMES', 'JURERE BEACH VILLAGE', 'APART HOTEL ROBERTO MONTEIRO', 'INGLESES PARK HOTEL', 'SLAVIERO ESSENTIAL INGLESES CONVENTION', 'IL CAMPANARIO VILLAGGIO RESORT', 'WK DESIGN HOTEL F

In [10]:
price_hotels_123m_fmtg = []

for price in list_price_hotels_123milhas:
    for dot in ["."]:
        price = price.replace(dot, "")
    price_hotels_123m_fmtg.append(price)
print(price_hotels_123m_fmtg)

['R$ 741', 'R$ 463', 'R$ 270', 'R$ 520', 'R$ 325', 'R$ 519', 'R$ 611', 'R$ 265', 'R$ 694', 'R$ 757', 'R$ 578', 'R$ 491', 'R$ 20114', 'R$ 443', 'R$ 822', 'R$ 850', 'R$ 500', 'R$ 366', 'R$ 771', 'R$ 389', 'R$ 832', 'R$ 372', 'R$ 398', 'R$ 536', 'R$ 524', 'R$ 765', 'R$ 437', 'R$ 1604', 'R$ 383', 'R$ 1472', 'R$ 341', 'R$ 1592', 'R$ 540', 'R$ 536', 'R$ 782', 'R$ 2286', 'R$ 1070', 'R$ 296', 'R$ 293', 'R$ 284', 'R$ 325', 'R$ 325', 'R$ 337', 'R$ 338', 'R$ 339', 'R$ 361', 'R$ 363', 'R$ 379', 'R$ 384', 'R$ 405', 'R$ 407', 'R$ 411', 'R$ 414', 'R$ 427', 'R$ 438', 'R$ 447', 'R$ 452', 'R$ 455', 'R$ 468', 'R$ 469', 'R$ 491', 'R$ 500', 'R$ 505', 'R$ 509', 'R$ 528', 'R$ 529', 'R$ 541', 'R$ 552', 'R$ 574', 'R$ 648', 'R$ 650', 'R$ 779', 'R$ 784', 'R$ 784', 'R$ 804', 'R$ 1093', 'R$ 17944', 'R$ 199', 'R$ 282', 'R$ 316', 'R$ 399', 'R$ 483', 'R$ 502', 'R$ 504', 'R$ 588', 'R$ 950', 'R$ 1093', 'R$ 2035', 'R$ 225', 'R$ 310', 'R$ 335', 'R$ 426', 'R$ 439', 'R$ 456', 'R$ 490', 'R$ 509', 'R$ 573', 'R$ 633', 'R$ 688

In [11]:
price_hotels_123m_fmtg2 = []

for price in price_hotels_123m_fmtg:
    for sign in ["R$ "]:
        price = price.replace(sign, "")
    price_hotels_123m_fmtg2.append(price)
print(price_hotels_123m_fmtg2)

['741', '463', '270', '520', '325', '519', '611', '265', '694', '757', '578', '491', '20114', '443', '822', '850', '500', '366', '771', '389', '832', '372', '398', '536', '524', '765', '437', '1604', '383', '1472', '341', '1592', '540', '536', '782', '2286', '1070', '296', '293', '284', '325', '325', '337', '338', '339', '361', '363', '379', '384', '405', '407', '411', '414', '427', '438', '447', '452', '455', '468', '469', '491', '500', '505', '509', '528', '529', '541', '552', '574', '648', '650', '779', '784', '784', '804', '1093', '17944', '199', '282', '316', '399', '483', '502', '504', '588', '950', '1093', '2035', '225', '310', '335', '426', '439', '456', '490', '509', '573', '633', '688', '885', '299', '313', '339', '346', '379', '386', '389', '416', '426', '429', '455', '468', '490', '538', '538', '631', '681', '785', '688', '885', '299', '313', '339', '346', '379', '386', '389', '416', '426', '429', '455', '468', '490', '538', '538', '631', '681', '785', '688', '885', '299', 

In [12]:
price_hotels_123m_fmtg2 = pd.to_numeric(price_hotels_123m_fmtg2, errors='coerce', downcast='unsigned')

In [13]:
data_hotels_123m = []

for elems in zip(name_hotels_123m_fmtg, price_hotels_123m_fmtg2):
    data_hotels_123m.append(elems)
data_hotels_123m_dict = dict(data_hotels_123m)
print(data_hotels_123m_dict)

{'COSTA NORTE PONTA DAS CANAS': 741, 'HOTEL PORTO SOL INGLESES': 463, 'HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE': 270, 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS': 520, 'HOTEL VALERIM FLORIANOPOLIS': 325, 'SLAVIERO BAIA NORTE FLORIANOPOLIS': 519, 'FAIAL PRIME SUITES': 611, 'CASTELMAR HOTEL': 265, 'BLUE TREE PREMIUM FLORIANOPOLIS': 694, 'NOVOTEL FLORIANOPOLIS': 757, 'CRIS HOTEL': 578, 'VILLAGE PARAISO': 491, 'LUMAR HOTEL - REDE ANDRADE': 20114, 'GENTIL HOTEL': 443, 'PALACE PRAIA HOTEL': 822, 'VILLAS JURERE HOTEL BOUTIQUE': 850, 'HOTEL INTERCITY FLORIANOPOLIS': 500, 'POUSADA BIZKAIA': 366, 'INGLESES PRAIA HOTEL': 771, 'PORTO DA ILHA HOTEL': 389, 'COSTA NORTE INGLESES': 832, 'INTERCLASS FLORIANOPOLIS': 372, 'IATE HOTEL CENTRO FLORIANOPOLIS': 398, 'RIO BRANCO APART HOTEL': 536, 'VARADERO PALACE HOTEL II': 524, 'HOTEL PORTO SOL BEACH': 765, 'MARIA DO MAR HOTEL': 437, 'HOTEL BOUTIQUE QUINTA DAS VIDEIRAS': 1604, 'OSCAR HOTEL': 383, 'HOTEL PALACE 1': 1472, 'POUSADA GOMES': 341, 'JURERE BEACH 

In [14]:
df_data_hotels_123m = pd.DataFrame(list(data_hotels_123m_dict.items()),
                   columns=['Hotel Name', 'Net Price 123 Milhas'])
display(df_data_hotels_123m)

,Hotel Name,Net Price 123 Milhas
0,COSTA NORTE PONTA DAS CANAS,741
1,HOTEL PORTO SOL INGLESES,463
2,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,270
3,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,520
4,HOTEL VALERIM FLORIANOPOLIS,325
...,...,...
113,INGLESES PARADISE,538
114,RESIDENCIAL LAS DUNAS,538
115,KASATO MARU APART HOTEL,631
116,PRAIA MOLE RESIDENCE,681


In [15]:
df_data_hotels_123m['Total Price 123 Milhas (w/ fee)'] = (df_data_hotels_123m['Net Price 123 Milhas'] * fee) + df_data_hotels_123m['Net Price 123 Milhas'].apply(pd.to_numeric) 

In [16]:
df_data_hotels_123m['Total Price 123 Milhas (w/ fee)'] = df_data_hotels_123m['Total Price 123 Milhas (w/ fee)'].astype(int)

In [17]:
df_data_hotels_123m

,Hotel Name,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee)
0,COSTA NORTE PONTA DAS CANAS,741,848
1,HOTEL PORTO SOL INGLESES,463,529
2,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,270,309
3,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,520,595
4,HOTEL VALERIM FLORIANOPOLIS,325,371
...,...,...,...
113,INGLESES PARADISE,538,615
114,RESIDENCIAL LAS DUNAS,538,615
115,KASATO MARU APART HOTEL,631,722
116,PRAIA MOLE RESIDENCE,681,779


In [18]:
length_df1 = len(df_data_hotels_123m)
length_df2 = len(list_channel_hotels_123milhas)
diff_length_dfs = length_df2 - length_df1
list_channel_hotels_123milhas = list_channel_hotels_123milhas[:length_df2 - diff_length_dfs]

In [19]:
df_data_hotels_123m.insert(loc=1, column='Omnibees Provider 123 Milhas', value=list_channel_hotels_123milhas)

In [20]:
df_data_hotels_123m

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee)
0,COSTA NORTE PONTA DAS CANAS,0,741,848
1,HOTEL PORTO SOL INGLESES,0,463,529
2,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,0,270,309
3,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,1,520,595
4,HOTEL VALERIM FLORIANOPOLIS,0,325,371
...,...,...,...,...
113,INGLESES PARADISE,0,538,615
114,RESIDENCIAL LAS DUNAS,0,538,615
115,KASATO MARU APART HOTEL,0,631,722
116,PRAIA MOLE RESIDENCE,0,681,779


In [21]:
copy_df_123m = df_data_hotels_123m.copy()

In [22]:
from datetime import datetime

checkin_decolar = datetime.strptime(checkin, "%d-%m-%Y").strftime('%Y-%m-%d')
checkout_decolar = datetime.strptime(checkout, "%d-%m-%Y").strftime('%Y-%m-%d')

In [23]:
# Scraper Hotel Prices Decolar

list_name_hotels_decolar = []
list_price_hotels_decolar = []

class ScraperDecolar:

    def scraper(self):
        self.data_collect()

    def data_collect(self): 
        page = 1      
        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://www.decolar.com/accommodations/results/CIT_{id_destination_decolar}/{checkin_decolar}/{checkout_decolar}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}'
        self.list_names = []
        self.list_prices = []
        self.driver.get(self.link)
        self.driver.find_element(By.XPATH, '//*[@id="lgpd-banner"]/div/a[2]').click()
        sleep(3)
        
        for p in range(20): # Number of pages to browse (nº > total, run through all)
            item = 1
            list_length = self.driver.find_elements(By.CLASS_NAME, 'accommodation-name')
            list_max = len(list_length)
            for i in range(list_max):
                c = 1
                while c < list_max:
                    try:
                        names_temp = self.driver.find_elements(By.XPATH,
                            f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[1]/div/div[2]/div/aloha-cluster-accommodation-info-container/div[1]/span')
                        self.list_names.append(names_temp[0].text)
                        sleep(1)
                        prices_temp = self.driver.find_elements(By.XPATH,
                            f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[2]/aloha-cluster-pricebox-container/div/div[2]/div[1]/aloha-price-container/aloha-summary-price/div/span[2]')
                        self.list_prices.append(prices_temp[0].text)          
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1                        
            try:
                page += 1
                self.driver.get(f'https://www.decolar.com/accommodations/results/CIT_{id_destination_decolar}/{checkin_decolar}/{checkout_decolar}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}')
                print(f'\u001b[32m{"Browsing..."}\u001b[0m')
                sleep(5)
            except:
                self.driver.quit()
        
        print("Raw data:")
        print(self.list_names)
        print(self.list_prices)

        for name in self.list_names:
            list_name_hotels_decolar.append(name)
        for price in self.list_prices:
            list_price_hotels_decolar.append(price)


start = ScraperDecolar()
start.scraper()


Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Raw data:
['Ingleses Park Hotel', 'Hotel São Sebastião da Praia', 'Hotel & Pousada Favareto', 'Hotel Porto Sol Beach', 'Jurerê Beach Village', 'Intercity Florianópolis', 'Hotel Porto Sol Ingleses', 'Das Nações Hotel', 'Rede Andrade Cecomtur', 'Majestic Palace', 'Hotel Ingleses Holiday', 'Palace Praia Hotel', 'Varadero Palace Hotel II', 'Costa Norte Ingleses', 'Hotel Praia Brava', 'Belluno Apart Hotel', 'Pousada Vó Regina', 'Interclass Florianópolis', 'WK Design Hotel Florianópolis', 'Village Paraíso', 'Hotel Monteiro Canasvieiras', 'Palm Beach Apart Hotel', 'Novotel Florianópolis', 'Green Mountain Pousada', 'Blue Tree Premium Florianópolis', 'IL Campanario Villaggio Resort', 'Mar e Mar Florianópolis', 'Castelmar Hotel', 'Hotel Vila Mar', 'Gaivotas Pr

In [24]:
# Data cleaning

name_hotels_desp_temp = list(pd.Series(list_name_hotels_decolar).str.upper())
name_hotels_desp_fmtg = []

for name in name_hotels_desp_temp:
    name_hotels_desp_fmtg.append(unidecode(name))
print(name_hotels_desp_fmtg)

['INGLESES PARK HOTEL', 'HOTEL SAO SEBASTIAO DA PRAIA', 'HOTEL & POUSADA FAVARETO', 'HOTEL PORTO SOL BEACH', 'JURERE BEACH VILLAGE', 'INTERCITY FLORIANOPOLIS', 'HOTEL PORTO SOL INGLESES', 'DAS NACOES HOTEL', 'REDE ANDRADE CECOMTUR', 'MAJESTIC PALACE', 'HOTEL INGLESES HOLIDAY', 'PALACE PRAIA HOTEL', 'VARADERO PALACE HOTEL II', 'COSTA NORTE INGLESES', 'HOTEL PRAIA BRAVA', 'BELLUNO APART HOTEL', 'POUSADA VO REGINA', 'INTERCLASS FLORIANOPOLIS', 'WK DESIGN HOTEL FLORIANOPOLIS', 'VILLAGE PARAISO', 'HOTEL MONTEIRO CANASVIEIRAS', 'PALM BEACH APART HOTEL', 'NOVOTEL FLORIANOPOLIS', 'GREEN MOUNTAIN POUSADA', 'BLUE TREE PREMIUM FLORIANOPOLIS', 'IL CAMPANARIO VILLAGGIO RESORT', 'MAR E MAR FLORIANOPOLIS', 'CASTELMAR HOTEL', 'HOTEL VILA MAR', 'GAIVOTAS PRAIA HOTEL', 'COSTA NORTE PONTA DAS CANAS', 'LETTO HOTEL JURERE GUEST HOUSE', 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS', 'HOTEL FAROL DA ILHA', 'GECKOS HOSTEL', 'SLAVIERO ESSENTIAL INGLESES CONVENTION', 'INNBOX - CANASVIEIRAS', 'ELITE HOTEL', 'VILLAS 

In [25]:
price_hotels_desp_fmtg = []

for price in list_price_hotels_decolar:
    for dot in ["."]:
        price = price.replace(dot, "")
    price_hotels_desp_fmtg.append(price)
print(price_hotels_desp_fmtg)

['892', '692', '736', '1166', '2130', '718', '612', '502', '362', '822', '648', '1100', '706', '1134', '704', '468', '398', '532', '1500', '804', '764', '400', '1080', '382', '1026', '3058', '642', '382', '502', '558', '992', '1320', '772', '534', '322', '1100', '468', '502', '1060', '600', '514', '628', '420', '630', '724', '462', '596', '472', '554', '430', '650', '554', '532', '520', '482', '462', '622', '482', '572', '854', '1388', '1050', '342', '1072', '626', '582', '640', '404', '452', '404', '542', '422', '648', '602', '2800', '502', '598', '642', '324', '636', '668', '492', '530', '416', '838', '25638', '558', '772', '308', '410', '498', '802', '712', '604', '682', '850', '262', '344', '544', '680', '460', '322', '452', '704', '762', '718', '674', '222', '564', '492', '524', '390', '1164', '1462', '248', '282', '1304', '680', '478', '1276', '340', '718', '402', '702', '442', '650', '522', '302', '508', '400', '654', '438', '272', '902', '592', '804', '676', '616', '798', '1602

In [26]:
price_hotels_desp_fmtg = pd.to_numeric(price_hotels_desp_fmtg, errors='coerce')

In [27]:
data_hotels_desp = []

for elems in zip(name_hotels_desp_fmtg, price_hotels_desp_fmtg):
    data_hotels_desp.append(elems)
data_hotels_desp_dict = dict(data_hotels_desp)
print(data_hotels_desp_dict)

{'INGLESES PARK HOTEL': 892, 'HOTEL SAO SEBASTIAO DA PRAIA': 692, 'HOTEL & POUSADA FAVARETO': 736, 'HOTEL PORTO SOL BEACH': 1166, 'JURERE BEACH VILLAGE': 2130, 'INTERCITY FLORIANOPOLIS': 718, 'HOTEL PORTO SOL INGLESES': 612, 'DAS NACOES HOTEL': 502, 'REDE ANDRADE CECOMTUR': 362, 'MAJESTIC PALACE': 822, 'HOTEL INGLESES HOLIDAY': 648, 'PALACE PRAIA HOTEL': 1100, 'VARADERO PALACE HOTEL II': 706, 'COSTA NORTE INGLESES': 1134, 'HOTEL PRAIA BRAVA': 704, 'BELLUNO APART HOTEL': 468, 'POUSADA VO REGINA': 398, 'INTERCLASS FLORIANOPOLIS': 532, 'WK DESIGN HOTEL FLORIANOPOLIS': 1500, 'VILLAGE PARAISO': 804, 'HOTEL MONTEIRO CANASVIEIRAS': 764, 'PALM BEACH APART HOTEL': 400, 'NOVOTEL FLORIANOPOLIS': 1080, 'GREEN MOUNTAIN POUSADA': 382, 'BLUE TREE PREMIUM FLORIANOPOLIS': 1026, 'IL CAMPANARIO VILLAGGIO RESORT': 3058, 'MAR E MAR FLORIANOPOLIS': 642, 'CASTELMAR HOTEL': 382, 'HOTEL VILA MAR': 502, 'GAIVOTAS PRAIA HOTEL': 558, 'COSTA NORTE PONTA DAS CANAS': 992, 'LETTO HOTEL JURERE GUEST HOUSE': 1320, 'HOT

In [28]:
df_data_hotels_desp = pd.DataFrame(list(data_hotels_desp_dict.items()),
                   columns=['Hotel Name', 'Total Price Decolar'])
display(df_data_hotels_desp)

,Hotel Name,Total Price Decolar
0,INGLESES PARK HOTEL,892
1,HOTEL SAO SEBASTIAO DA PRAIA,692
2,HOTEL & POUSADA FAVARETO,736
3,HOTEL PORTO SOL BEACH,1166
4,JURERE BEACH VILLAGE,2130
...,...,...
224,RED MOON BUNGALOW,544
225,CASA DO RIO VERMELHO- WITH JACUZZI,2148
226,FLAT KAKI IN NEW CAMPECHE SOUTH ISLAND,702
227,NET KIT FOR 4 PEOPLE,340


In [29]:
copy_df_desp = df_data_hotels_desp.copy()

In [30]:
# Data merge

data_merge = pd.merge(df_data_hotels_123m, df_data_hotels_desp, on=['Hotel Name'], how='left')
display(data_merge)

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee),Total Price Decolar
0,COSTA NORTE PONTA DAS CANAS,0,741,848,992.0
1,HOTEL PORTO SOL INGLESES,0,463,529,612.0
2,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,0,270,309,NaN
3,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,1,520,595,772.0
4,HOTEL VALERIM FLORIANOPOLIS,0,325,371,430.0
...,...,...,...,...,...
113,INGLESES PARADISE,0,538,615,NaN
114,RESIDENCIAL LAS DUNAS,0,538,615,630.0
115,KASATO MARU APART HOTEL,0,631,722,NaN
116,PRAIA MOLE RESIDENCE,0,681,779,NaN


In [31]:
df_rate_comparison = data_merge.dropna()
df_rate_comparison.shape

(72, 5)

In [32]:
pd.options.mode.chained_assignment = None

for prices in df_rate_comparison:
    diff = (df_rate_comparison.loc[:, 'Total Price Decolar'] / df_rate_comparison.loc[:, 'Total Price 123 Milhas (w/ fee)'] -1 ) * 100
    df_rate_comparison.loc[:, '% Variation (total prices)'] = round(diff)
display(df_rate_comparison)

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee),Total Price Decolar,% Variation (total prices)
0,COSTA NORTE PONTA DAS CANAS,0,741,848,992.0,17.0
1,HOTEL PORTO SOL INGLESES,0,463,529,612.0,16.0
3,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,1,520,595,772.0,30.0
4,HOTEL VALERIM FLORIANOPOLIS,0,325,371,430.0,16.0
5,SLAVIERO BAIA NORTE FLORIANOPOLIS,1,519,594,596.0,0.0
...,...,...,...,...,...,...
104,CASTELO DA ILHA,0,379,433,564.0,30.0
106,INNBOX - CANASVIEIRAS,0,389,445,468.0,5.0
107,SEA WOLF SURF CLUB,0,416,476,508.0,7.0
114,RESIDENCIAL LAS DUNAS,0,538,615,630.0,2.0


In [33]:
# Summary

import math

measurements = 0
for m in df_rate_comparison['% Variation (total prices)']:
  if not math.isnan(m):
    measurements += 1
print(f'Total measurements: {measurements}')

unfavorable = 0
for adv in df_rate_comparison['% Variation (total prices)']:
    if adv >= 1:
        unfavorable += 1
percent_unfavorable = (unfavorable / measurements) * 100
print(f'Percentage of unfavorable measurements: {percent_unfavorable:.2f}%')

Total measurements: 72
Percentage of unfavorable measurements: 91.67%


In [34]:
# Highlights

def highlight(s):
    color_red = 'red'
    color_yellow = 'yellow'
    color_green = 'green'
    if s >= 1:
      return 'background-color: %s' % color_red
    elif s < 1 and s > -1:
      return 'background-color: %s' % color_yellow
    elif s <= -1:
      return 'background-color: %s' % color_green

report = df_rate_comparison.style.applymap(highlight, subset=pd.IndexSlice[:, ['% Variation (total prices)']])

In [35]:
# Dataframe to XLSX

writer = pd.ExcelWriter('Rate-parity-report-123milhas-Decolar-{}.xlsx'.format(name_destination))
report.to_excel(writer, f'IN_{checkin}-OUT_{checkout}', index=False)
writer.save()

In [36]:
# Download XLSX

from google.colab import files

files.download('Rate-parity-report-123milhas-Decolar-{}.xlsx'.format(name_destination))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>